In [1]:
import zipfile
from pathlib import Path

zip_path = Path("data/Preprocesed Train Data Side camera.zip")
extract_path = Path("data/custom_data")

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [2]:
!python train_val_split.py --datapath="data/custom_data/Preprocesed Data" --train_pct=0.8

Number of image files: 1350
Number of annotation files: 1350
Images moving to train: 1080
Images moving to validation: 270


In [3]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': 'data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = 'data/custom_data/Preprocesed Data/classes.txt'
path_to_data_yaml = 'data/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')

file_path = Path("data/data.yaml")

with open(file_path, "r", encoding="utf-8") as f:
    print(f.read())

Created config file at data/data.yaml

File contents:

path: data
train: train/images
val: validation/images
nc: 11
names:
- 01_Good
- 10_Overhang_Sag
- 11_Shifted_Layer
- 12_Warping
- 13_Delamination
- 14_Unclear
- 15_Not_Printing
- 4_Foreign_Body
- 6_Stringing
- 7_Spaghetti
- 9_Poor_Bridging



In [5]:
yaml_text = """path: data
train: train/images
val: validation/images
test: test/images

nc: 11
names:
  0: 01_Good
  1: 4_Foreign_Body
  2: 6_Stringing
  3: 7_Spaghetti
  4: 9_Poor_Bridging
  5: 10_Overhang_Sag
  6: 11_Shifted_Layer
  7: 12_Warping
  8: 13_Delamination
  9: 14_Unclear
  10: 15_Not_Printing
"""

Path("data/data.yaml").write_text(yaml_text, encoding="utf-8")
print(Path("data/data.yaml").read_text(encoding="utf-8"))

path: data
train: train/images
val: validation/images
test: test/images

nc: 11
names:
  0: 01_Good
  1: 4_Foreign_Body
  2: 6_Stringing
  3: 7_Spaghetti
  4: 9_Poor_Bridging
  5: 10_Overhang_Sag
  6: 11_Shifted_Layer
  7: 12_Warping
  8: 13_Delamination
  9: 14_Unclear
  10: 15_Not_Printing



In [6]:
from pathlib import Path
from collections import Counter

root = Path("data")
label_files = list(root.rglob("labels/*.txt")) + list(root.rglob("**/labels/**/*.txt"))
print("Found label files:", len(label_files))

counts = Counter()
max_id = -1

for f in label_files:
    for line in f.read_text(encoding="utf-8", errors="ignore").splitlines():
        if not line.strip():
            continue
        cls = int(float(line.split()[0]))
        counts[cls] += 1
        max_id = max(max_id, cls)

print("Class IDs present:", sorted(counts.keys()))
print("Max class ID:", max_id)
print("Counts:", dict(sorted(counts.items())))

Found label files: 6272
Class IDs present: [1, 2, 3, 4, 5, 6, 7, 8]
Max class ID: 8
Counts: {1: 2070, 2: 1118, 3: 1214, 4: 836, 5: 2492, 6: 1064, 7: 1326, 8: 3126}


In [7]:
import sys, torch
import ultralytics

print("Python exe:", sys.executable)
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)
print("Ultralytics version:", ultralytics.__version__)

Python exe: c:\Program Files\Python312\python.exe
Torch version: 2.5.1+cu121
CUDA available: True
CUDA device count: 1
CUDA device name: NVIDIA GeForce RTX 4060 Laptop GPU
Ultralytics version: 8.3.229


Hyperparameter tuning with Grid-Search

In [1]:
from ultralytics import YOLO
from pathlib import Path
import itertools
import pandas as pd
import time

# CONFIG
DATA_YAML = "data/data.yaml"
BASE_MODEL = "yolov8s.pt"          
DEVICE = 0                        
PROJECT = "runs/gridsearch"       
GRID_EPOCHS = 10
FIXED_IMGSZ = 640                        
SEED = 42

# GRID SETUP
grid = {
    "lr0": [0.01, 0.001],
    "weight_decay": [0.0005, 0.005, 0.05],
    "optimizer": ["SGD", "AdamW"],
    "box": [7.5, 10]
}

Path(PROJECT).mkdir(parents=True, exist_ok=True)

keys = list(grid.keys())
combos = list(itertools.product(*[grid[k] for k in keys]))
print(f"Total runs: {len(combos)}")

rows = []
t0 = time.time()

for i, values in enumerate(combos, start=1):
    params = dict(zip(keys, values))
    run_name = "gs_" + "_".join([f"{k}={v}" for k, v in params.items()])

    print(f"\n[{i}/{len(combos)}] Training: {run_name}")
    model = YOLO(BASE_MODEL)

    train_results = model.train(
        data=DATA_YAML,
        epochs=GRID_EPOCHS,
        imgsz=FIXED_IMGSZ,
        lr0=params["lr0"],
        weight_decay=params["weight_decay"],
        optimizer=params["optimizer"],
        box=params["box"],
        batch=16,
        device=DEVICE,
        seed=SEED,
        deterministic=True,
        project=PROJECT,
        name=run_name,
        plots=False,           # faster
        save=True,
        val=True,
        patience=4            # early stopping helps speed grid search
    )

# After training, evaluate the best checkpoint on val to get a clean final score
    best_path = Path(model.trainer.save_dir) / "weights" / "best.pt"
    best_model = YOLO(str(best_path))
    val_metrics = best_model.val(data=DATA_YAML, split="val", device=DEVICE, plots=False)

    # Extract metrics (these attribute names are stable in Ultralytics v8.x)
    row = {
        "run_name": run_name,
        "save_dir": str(model.trainer.save_dir),
        "best_pt": str(best_path),
        **params,
        "imgsz": params.get("imgsz", FIXED_IMGSZ),
        "val_mAP50_95": float(val_metrics.box.map),     # mAP@[.5:.95]
        "val_mAP50": float(val_metrics.box.map50),      # mAP@.5
        "val_precision": float(val_metrics.box.mp),     # mean precision
        "val_recall": float(val_metrics.box.mr),        # mean recall
    }
    rows.append(row)

 # Write partial results continuously (so you don’t lose progress)
    df = pd.DataFrame(rows).sort_values("val_mAP50_95", ascending=False)
    df.to_csv(Path(PROJECT) / "grid_results.csv", index=False)

print(f"\nDone in {(time.time()-t0)/60:.1f} minutes.")
df.head(10)

Total runs: 24

[1/24] Training: gs_lr0=0.01_weight_decay=0.0005_optimizer=SGD_box=7.5
Ultralytics 8.3.229  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=gs_lr0=0.01_weight_decay=0.0005_optimizer=SGD

,run_name,save_dir,best_pt,lr0,weight_decay,optimizer,box,imgsz,val_mAP50_95,val_mAP50,val_precision,val_recall
22,gs_lr0=0.001_weight_decay=0.05_optimizer=AdamW...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0500,AdamW,7.5,640,0.498399,0.864042,0.829144,0.788562
14,gs_lr0=0.001_weight_decay=0.0005_optimizer=Ada...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0005,AdamW,7.5,640,0.494280,0.853855,0.833415,0.800081
19,gs_lr0=0.001_weight_decay=0.005_optimizer=Adam...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0050,AdamW,10.0,640,0.493097,0.842526,0.808310,0.791973
23,gs_lr0=0.001_weight_decay=0.05_optimizer=AdamW...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0500,AdamW,10.0,640,0.489546,0.853101,0.812820,0.791075
0,gs_lr0=0.01_weight_decay=0.0005_optimizer=SGD_...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.010,0.0005,SGD,7.5,640,0.488617,0.845222,0.816380,0.793783
5,gs_lr0=0.01_weight_decay=0.005_optimizer=SGD_b...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.010,0.0050,SGD,10.0,640,0.487127,0.836688,0.785880,0.791272
15,gs_lr0=0.001_weight_decay=0.0005_optimizer=Ada...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0005,AdamW,10.0,640,0.484874,0.836542,0.786715,0.788674
18,gs_lr0=0.001_weight_decay=0.005_optimizer=Adam...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.001,0.0050,AdamW,7.5,640,0.483735,0.845245,0.835705,0.800545
1,gs_lr0=0.01_weight_decay=0.0005_optimizer=SGD_...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.010,0.0005,SGD,10.0,640,0.483250,0.836670,0.791100,0.819185
4,gs_lr0=0.01_weight_decay=0.005_optimizer=SGD_b...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,C:\Users\User\Documents\GitHub\Real-Time-Defec...,0.010,0.0050,SGD,7.5,640,0.469141,0.823696,0.796741,0.783759


In [ ]:
from ultralytics import YOLO

PROJECT = "runs/gridsearch"

df = pd.read_csv(Path(PROJECT) / "grid_results.csv").sort_values("val_mAP50_95", ascending=False)
best = df.iloc[0].to_dict()
print("Best config:", best)

final_name = "best_hyperparameters"
model = YOLO(BASE_MODEL)
model.train(data="data/data.yaml",
            epochs=60,
            imgsz=FIXED_IMGSZ,
            lr0=float(best["lr0"]),
            weight_decay=float(best["weight_decay"]),
            optimizer=str(best["optimizer"]),
            box=float(best["box"]),
            batch=16,
            device=DEVICE,
            seed=SEED,
            deterministic=True,
            project="runs/detect",
            name=final_name,
            plots=True,
            val=True
)

best_path = Path(model.trainer.save_dir) / "weights" / "best.pt"
last_path = Path(model.trainer.save_dir) / "weights" / "last.pt"
print("✅ Best checkpoint:", best_path)
print("✅ Last checkpoint:", last_path)

Ultralytics 8.3.229  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001F7B25E6D80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [ ]:
model = YOLO("runs/detect/train/weights/best.pt")

In [ ]:
model.predict(source="data/validation/images",
              save=True
)

print("✅ Saved validation predictions under runs/detect/predict*")

Ultralytics 8.3.229  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 11,129,841 parameters, 0 gradients, 28.5 GFLOPs

image 1/270 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\validation\images\00a1e045-233photo_2024-08-21T16_37_51-191689.jpg: 416x640 3 12_Warpings, 39.9ms
image 2/270 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\validation\images\00b9f800-left_163photo_2024-10-02T14_05_41-353505.jpg: 416x640 1 6_Stringing, 4.4ms
image 3/270 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\validation\images\0326804c-left_273photo_2024-10-02T14_51_55-743754.jpg: 416x640 5 10_Overhang_Sags, 4.3ms
image 4/270 c:\Users\User\Documents\GitHub\Real-Time-Defect-Detection-Capstone-Project\data\validation\images\0372d085-right_photo_66_2024-11-21_17-43-43.jpg: 416x640 3 4_Foreign_Bodys, 4.9ms
image 5/270 c:\Users\User\Documents\

In [ ]:
model.predict(
    source="data/test/images",  # wherever your test images are
    save=True,
    save_txt=True,     # saves YOLO-format predictions (class x y w h conf) per image
    save_conf=True
)

In [ ]:
model.val(data="data/data.yaml",
          split="test",
          device=0
)

print("✅ Test evaluation complete.")